In [97]:
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import os, json
import glob
from pandas.io.json import json_normalize
from sklearn.metrics import accuracy_score
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import TransformedTargetRegressor, ColumnTransformer
from pandas.io.json import json_normalize
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import VarianceThreshold

In [71]:
y_train = pd.read_csv('train_winners.csv')

In [72]:
f = open('train_timelines/train_timelines/timeline_{matchIds}.json'.format(matchIds = 1091))

data = json.load(f)

In [73]:
df = []

In [74]:
for matchId in range(0, 8000):
    f = open('train_timelines/train_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    df.append({
        'matchId' : matchId,
        'blueGold' : (
            data['frames'][-1]['participantFrames']['1']['totalGold'] +
            data['frames'][-1]['participantFrames']['2']['totalGold'] +
            data['frames'][-1]['participantFrames']['3']['totalGold'] +
            data['frames'][-1]['participantFrames']['4']['totalGold'] +
            data['frames'][-1]['participantFrames']['5']['totalGold']
        ),
        'blueXP' : (
            data['frames'][-1]['participantFrames']['1']['xp'] +
            data['frames'][-1]['participantFrames']['2']['xp'] +
            data['frames'][-1]['participantFrames']['3']['xp'] +
            data['frames'][-1]['participantFrames']['4']['xp'] +
            data['frames'][-1]['participantFrames']['5']['xp']
        ),
        'redGold' : (
            data['frames'][-1]['participantFrames']['6']['totalGold'] + 
            data['frames'][-1]['participantFrames']['7']['totalGold'] +
            data['frames'][-1]['participantFrames']['8']['totalGold'] +
            data['frames'][-1]['participantFrames']['9']['totalGold'] +
            data['frames'][-1]['participantFrames']['10']['totalGold']
        ),
        'redXP' : (
            data['frames'][-1]['participantFrames']['6']['xp'] + 
            data['frames'][-1]['participantFrames']['7']['xp'] +
            data['frames'][-1]['participantFrames']['8']['xp'] +
            data['frames'][-1]['participantFrames']['9']['xp'] +
            data['frames'][-1]['participantFrames']['10']['xp']
        )
    })

In [75]:
X_train = pd.DataFrame(df)

In [76]:
gold = ['blueGold', 'redGold']
xp = ['blueXP', 'redXP']
both = ['blueGold', 'redGold', 'blueXP', 'redXP']

In [77]:
logreg = LogisticRegression().fit(X_train[gold], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[gold]))

0.705375

In [78]:
logreg = LogisticRegression().fit(X_train[xp], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[xp]))

0.672

In [79]:
logreg = LogisticRegression().fit(X_train[both], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[both]))

0.70975

In [80]:
df = []

for matchId in range(8000, 10000):
    f = open('test_timelines/test_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    df.append({
        'matchId' : matchId,
        'blueGold' : (
            data['frames'][-1]['participantFrames']['1']['totalGold'] +
            data['frames'][-1]['participantFrames']['2']['totalGold'] +
            data['frames'][-1]['participantFrames']['3']['totalGold'] +
            data['frames'][-1]['participantFrames']['4']['totalGold'] +
            data['frames'][-1]['participantFrames']['5']['totalGold']
        ),
        'blueXP' : (
            data['frames'][-1]['participantFrames']['1']['xp'] +
            data['frames'][-1]['participantFrames']['2']['xp'] +
            data['frames'][-1]['participantFrames']['3']['xp'] +
            data['frames'][-1]['participantFrames']['4']['xp'] +
            data['frames'][-1]['participantFrames']['5']['xp']
        ),
        'redGold' : (
            data['frames'][-1]['participantFrames']['6']['totalGold'] + 
            data['frames'][-1]['participantFrames']['7']['totalGold'] +
            data['frames'][-1]['participantFrames']['8']['totalGold'] +
            data['frames'][-1]['participantFrames']['9']['totalGold'] +
            data['frames'][-1]['participantFrames']['10']['totalGold']
        ),
        'redXP' : (
            data['frames'][-1]['participantFrames']['6']['xp'] + 
            data['frames'][-1]['participantFrames']['7']['xp'] +
            data['frames'][-1]['participantFrames']['8']['xp'] +
            data['frames'][-1]['participantFrames']['9']['xp'] +
            data['frames'][-1]['participantFrames']['10']['xp']
        )
    })
    
X_test = pd.DataFrame(df)

In [81]:
submissionV4 = X_test[['matchId']]
submissionV4 = submissionV4.copy()
submissionV4['win'] = logreg.predict(X_test[both])
submissionV4['winner'] = np.where(submissionV4['win'] == True, 100, 200)
submissionV4 = submissionV4[['matchId', 'winner']]

In [82]:
X_train['goldDifference'] = X_train['blueGold'] - X_train['redGold']

X_train['xpDifference'] = X_train['blueXP'] - X_train['redXP']

gold = ['goldDifference']
xp = ['xpDifference']
both = ['goldDifference', 'xpDifference']

In [83]:
logreg = LogisticRegression().fit(X_train[gold], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[gold]))

0.7055

In [84]:
logreg = LogisticRegression().fit(X_train[xp], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[xp]))

0.6715

In [85]:
logreg = LogisticRegression().fit(X_train[both], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[both]))

0.70825

In [87]:
df = []

for matchId in range(0, 8000):
    f = open('train_timelines/train_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    for frame in range(len(data['frames']) - 1, -1, -1):
        for i in range(0, len(data['frames'][frame]['events'])):
            if data['frames'][frame]['events'][i]['type']  == 'CHAMPION_KILL':
                if data['frames'][frame]['events'][i]['killerId'] < 6:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 1,
                        'redKills' : 0

                    })
                else:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 0,
                        'redKills' : 1

                    })
            else:
                pass

killsTrain = pd.DataFrame(df).groupby('matchId').sum().reset_index()

In [88]:
X_train = X_train.merge(right = killsTrain, on = ['matchId'], how = 'left').fillna(0)

kills = ['blueGold', 'redGold', 'blueXP', 'redXP', 'blueKills', 'redKills']

In [90]:
logreg = LogisticRegression().fit(X_train[kills], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[kills]))

0.70975

In [91]:
df = []

for matchId in range(0, 8000):
    f = open('train_timelines/train_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    for frame in range(len(data['frames']) - 1, -1, -1):
        for i in range(0, len(data['frames'][frame]['events'])):
            if data['frames'][frame]['events'][i]['type']  == 'ELITE_MONSTER_KILL':
                if data['frames'][frame]['events'][i]['monsterType']  == 'DRAGON':
                    if data['frames'][frame]['events'][i]['killerTeamId'] == 100:
                        df.append({
                            'matchId' : matchId,
                            'blueDragons' : 1,
                            'redDragons' : 0
                        })
                    else:
                        df.append({
                            'matchId' : matchId,
                            'blueDragons' : 0,
                            'redDragons' : 1
                        })
                else:
                    pass
            else:
                pass

dragonsTrain = pd.DataFrame(df).groupby('matchId').sum().reset_index()

In [92]:
X_train = X_train.merge(right = dragonsTrain, on = ['matchId'], how = 'left').fillna(0)

dragons = ['blueGold', 'redGold', 'blueXP', 'redXP', 'blueKills', 'redKills', 'blueDragons', 'redDragons']

In [93]:
logreg = LogisticRegression().fit(X_train[dragons], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, logreg.predict(X_train[dragons]))

0.70975

In [98]:
pipe = Pipeline(
    steps = [
        ('pf', PolynomialFeatures(interaction_only = True, include_bias = False)),
        ('vt', VarianceThreshold()),
        ('scaler', StandardScaler()),
        ('logistic', LogisticRegression(max_iter = 10000))
    ]
)

pipe.fit(X_train[dragons], y_train['winner'] == 100)

Pipeline(steps=[('pf',
                 PolynomialFeatures(include_bias=False, interaction_only=True)),
                ('vt', VarianceThreshold()), ('scaler', StandardScaler()),
                ('logistic', LogisticRegression(max_iter=10000))])

In [99]:
accuracy_score(y_train['winner'] == 100, pipe.predict(X_train[dragons]))

0.71925

In [103]:
df = []

for matchId in range(8000, 10000):
    f = open('test_timelines/test_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    for frame in range(len(data['frames']) - 1, -1, -1):
        for i in range(0, len(data['frames'][frame]['events'])):
            if data['frames'][frame]['events'][i]['type']  == 'CHAMPION_KILL':
                if data['frames'][frame]['events'][i]['killerId'] < 6:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 1,
                        'redKills' : 0

                    })
                else:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 0,
                        'redKills' : 1

                    })
            else:
                pass

killsTest = pd.DataFrame(df).groupby('matchId').sum().reset_index()

In [104]:
X_test = X_test.merge(right = killsTest, on = ['matchId'], how = 'left').fillna(0)

In [106]:
df = []

for matchId in range(8000, 10000):
    f = open('test_timelines/test_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    for frame in range(len(data['frames']) - 1, -1, -1):
        for i in range(0, len(data['frames'][frame]['events'])):
            if data['frames'][frame]['events'][i]['type']  == 'ELITE_MONSTER_KILL':
                if data['frames'][frame]['events'][i]['monsterType']  == 'DRAGON':
                    if data['frames'][frame]['events'][i]['killerTeamId'] == 100:
                        df.append({
                            'matchId' : matchId,
                            'blueDragons' : 1,
                            'redDragons' : 0
                        })
                    else:
                        df.append({
                            'matchId' : matchId,
                            'blueDragons' : 0,
                            'redDragons' : 1
                        })
                else:
                    pass
            else:
                pass

dragonsTest = pd.DataFrame(df).groupby('matchId').sum().reset_index()

In [107]:
X_test = X_test.merge(right = dragonsTest, on = ['matchId'], how = 'left').fillna(0)

In [109]:
submissionV6 = X_test[['matchId']]
submissionV6 = submissionV6.copy()
submissionV6['win'] = pipe.predict(X_test[dragons])
submissionV6['winner'] = np.where(submissionV6['win'] == True, 100, 200)
submissionV6 = submissionV6[['matchId', 'winner']]
submissionV6.to_csv('submissionV6.csv', index = False)

In [101]:
df = []

for matchId in range(8000, 10000):
    f = open('test_timelines/test_timelines/timeline_{matchIds}.json'.format(matchIds = matchId))
    
    data = json.load(f)
    for frame in range(len(data['frames']) - 1, -1, -1):
        for i in range(0, len(data['frames'][frame]['events'])):
            if data['frames'][frame]['events'][i]['type']  == 'CHAMPION_KILL':
                if data['frames'][frame]['events'][i]['killerId'] < 6:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 1,
                        'redKills' : 0

                    })
                else:
                    df.append({
                        'matchId' : matchId,
                        'blueKills' : 0,
                        'redKills' : 1

                    })
            else:
                pass

killsTest = pd.DataFrame(df).groupby('matchId').sum().reset_index()

In [110]:
X_train.head()

,matchId,blueGold,blueXP,redGold,redXP,goldDifference,xpDifference,blueKills,redKills,blueDragons,redDragons
0,0,14146,17159,17539,18773,-3393,-1614,2.0,10.0,0.0,1.0
1,1,17662,17770,14816,18028,2846,-258,6.0,2.0,0.0,1.0
2,2,16374,16837,17347,17467,-973,-630,7.0,10.0,1.0,0.0
3,3,19247,17562,15145,16196,4102,1366,10.0,4.0,1.0,0.0
4,4,16776,18264,15385,17194,1391,1070,7.0,4.0,0.0,1.0


In [ ]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
knn = Pipeline(
    steps = [
        ('scale', StandardScaler()),
        ('knn', KNeighborsClassifier())
    ]
)

param_grid = {
    'knn__n_neighbors': [10, 25, 50, 100],
}

knn_gs = GridSearchCV(estimator = knn, param_grid = param_grid)

knn_gs.fit(X_train, y_train['winner'] == 100)

In [ ]:
knn_gs.best_params_

In [ ]:
clf_3 = KNeighborsClassifier(n_neighbors=100)


In [ ]:
from sklearn.tree import DecisionTreeRegressor

In [ ]:
dt = Pipeline(
    steps = [
        ('dtr', DecisionTreeRegressor())
    ]
)
    
dt.fit(X_train, y_train)

In [ ]:
accuracy_score(y_train['winner'] == 100, dt.predict(X_train))

In [ ]:
from sklearn.ensemble import RandomForestRegressor

In [ ]:
rf = RandomForestRegressor().fit(X_train[xp], y_train['winner'] == 100)

accuracy_score(y_train['winner'] == 100, rf.predict(X_train[xp]))

In [ ]:
print(f'R2 Score: {r2_score(X_test, rf.predict(X_test))}')
print(f'MAE: {mean_absolute_error(X_test, rf.predict(X_test))}')